# 23. Namespaces

Namespaces were mentioned at the end of the notebook on Variables and the notebook on Importing Modules. We know that a namespace or a context is a naming system for Python objects. It can be thought of as a dictionary structure where any object can be stored. Everything in Python is an object and modules can be imported within other modules.

Where does the namespace come from? When a Python script is executed, Python defines some special variables, and one of them is `__name__`, for namespace. The namespace when a script is execued (ie. `python script.py`) is called `__main__`, the main Python module.

In [ ]:
print(__name__)

When code is imported from another module, code from that module is assigned to the current namespace in a variable with the module's name or an alias. The module's name will be set by Python as its `__name__`.

For example, to access the `os` module, we import the module into the current namespace. We can then access the module locally in the `os` namespace.

In [ ]:
import os
os.__name__

The variables assigned within the `__main__` context can be accessed using the built-in `globals()` function.

In [ ]:
globals()

In `__main__`, the `globals()` are also the `locals()`.

In [ ]:
globals() == locals()

Functions, among other things in Python, also have their own enclosed context. The variables in the current execution context can be accessed using the built-in `locals()` function.

In [ ]:
def func():
    return globals() == locals()

func()

In [ ]:
def func():
    this_var = 'enclosed variable'
    return locals()

func()

In Pyhon 3.x, the `i` in the code below maintains its value outside of the list comprehension. In Python 2.x, the value of `i` changes to the last used value in the list comrehension.

In [ ]:
i = 0
[print(i) for i in range(3)]
i

## LEGB

Given that namespaces can be nested and the same variable names can be used in different namespaces, how does Python get the right variable from the right namespace?

For example, what will be the output of the code below?

In [ ]:
x = 0
def outer():
    x = 1
    def inner():
        x = 2
        print("inner:", x)

    inner()
    print("outer:", x)

outer()
print("global:", x)

Python has a scope resolution order that uses the __LEGB__-rule:

### Local -> Enclosed -> Global -> Built-in

When searching for variables, the local namespace is searched first.

If the variable is not found in the local namespace, the enclosing namespace will be searched next, if there is one.

It goes up to search the global namespace and then the namespace for built-in variables.

Note that variables outside the current namespace can be read but not reassigned.

In [ ]:
i = 1
def read():
    print(i)

def reassign():
    i += 1

read()  # can read i
reassign()  # but there is no assignment of i in the local namespace

## [Rebinding variables](https://www.python.org/dev/peps/pep-3104/)

Python can rebind or reassign to variables in the global scope or declare that a variable is nonlocal. In Python 2.x, the `nonlocal` keyword is not available.

Check the effects of the declarations below:

In [ ]:
x = 0
def outer():
    x = 1
    def inner():
        nonlocal x  # declare nonlocal
        x = 2
        print("inner:", x)

    inner()
    print("outer:", x)

outer()
print("global:", x)

In [ ]:
x = 0
def outer():
    x = 1
    def inner():
        global x  # declare global
        x = 2
        print("inner:", x)

    inner()
    print("outer:", x)

outer()
print("global:", x)

Even though variables can be reassigned, it is usually not a good idea to modify variables outside of your current scope. Consider passing variables as arguments rather than rebinding, then assign those returned values. This makes it clear and easier to debug.

## `from module import *`

In general, it's not a good idea to import everything. It is better to be explicit than implicit. In this case, importing and assigning variables in the current namespace. Import it into its own namespace.


## Default Keyword Arguments

Python's __default arguments are evaluated once when the function is defined__, not each time the function is called. This means that if you use a mutable default argument and mutate it, that object is already mutated for all future calls to the function.

In [ ]:
def mutable_default(element, default=[]):
    default.append(element)
    return default

mutable_default(1)

In [ ]:
mutable_default(2)
mutable_default(3)

For the desired behavior, default values should be assigned within the enclosed scope, not as keyword arguments in the function's definition. The value of the keyword argument should be assigned to __`None`__.

In [ ]:
def immutable_default(element, default=None):
    if default is None:
        default = []

    default.append(element)
    return default

immutable_default(1)

In [ ]:
immutable_default(2)
immutable_default(3)

## Late Binding Closures

Python's __closures are late binding__. This means that the values of variables used in closures are looked up at the time the inner function is called. It looks for the value when it is needed. When a returned function is called, the value returned is the value at call time (as opposed to when the function is defined, when default keyword arguments are provided). By then, based on the execution, the value is set as the final value.

In [ ]:
def create_multipliers():
    multipliers = []

    for i in range(5):
        def multiplier(x):
            return i * x
        multipliers.append(multiplier)

    return multipliers

for multiplier in create_multipliers():
    print(multiplier(2))

The output was different from expected:

```
0
2
4
6
8
```

Instead of the list of values above, the last value is instead repeated.

The solution is to apply a keyword argument that supplies a default value to a variable. This immediately creates a variable that is bound to the local scope and makes the closure [evaluate when the function is defined](./23.%20Namespaces.ipynb#Default-Keyword-Arguments) instead of its usual late binding behavior.

In [ ]:
def create_multipliers():
    multipliers = []

    for i in range(5):
        def multiplier(x, i=i):  # assign default value
            return i * x
        multipliers.append(multiplier)

    return multipliers

for multiplier in create_multipliers():
    print(multiplier(2))